In [4]:
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
import dask.dataframe as dd
from fastparquet import write
import fastparquet
import multiprocessing as mp
import snappy
import argparse
import os

In [11]:
pkl_files=glob.glob(f'/datanaco/dharp/compounds/datasets/googleV3/*pkl')
len(pkl_files)

pkl_files_dict = dict()
for cur_file in pkl_files: 
    fsize = os.stat(cur_file).st_size
    pkl_files_dict[cur_file] = fsize

In [15]:
size_df=pd.DataFrame.from_dict(pkl_files_dict,orient='index')
size_df.reset_index(inplace=True)
size_df.columns=['fname','fsize']
size_df

,fname,fsize
0,/datanaco/dharp/compounds/datasets/googleV3/13...,163270847
1,/datanaco/dharp/compounds/datasets/googleV3/15...,204232301
2,/datanaco/dharp/compounds/datasets/googleV3/16...,1489989387
3,/datanaco/dharp/compounds/datasets/googleV3/14...,688697234
4,/datanaco/dharp/compounds/datasets/googleV3/16...,154155010
...,...,...
6525,/datanaco/dharp/compounds/datasets/googleV3/17...,272869646
6526,/datanaco/dharp/compounds/datasets/googleV3/80...,5136895830
6527,/datanaco/dharp/compounds/datasets/googleV3/14...,1735814885
6528,/datanaco/dharp/compounds/datasets/googleV3/19...,109631530


In [26]:
to_divide_list=size_df.sort_values(ascending=False,by='fsize',ignore_index=True)
to_divide_list['row_perc']=to_divide_list.fsize/to_divide_list.fsize.sum()*100
to_divide_list

,fname,fsize,row_perc
0,/datanaco/dharp/compounds/datasets/googleV3/80...,7677431026,1.550286e-01
1,/datanaco/dharp/compounds/datasets/googleV3/79...,7228628958,1.459660e-01
2,/datanaco/dharp/compounds/datasets/googleV3/80...,7134667418,1.440687e-01
3,/datanaco/dharp/compounds/datasets/googleV3/14...,7076838209,1.429009e-01
4,/datanaco/dharp/compounds/datasets/googleV3/81...,7050167074,1.423624e-01
...,...,...,...
6525,/datanaco/dharp/compounds/datasets/googleV3/16...,19672,3.972321e-07
6526,/datanaco/dharp/compounds/datasets/googleV3/13...,18182,3.671449e-07
6527,/datanaco/dharp/compounds/datasets/googleV3/16...,11446,2.311264e-07
6528,/datanaco/dharp/compounds/datasets/googleV3/22...,10357,2.091365e-07


In [27]:
maxvalue = 48*10**9

lastvalue = 0
newcum = []
labels=[]
cur_label=1
for row in to_divide_list.itertuples():
    thisvalue =  row.fsize + lastvalue
    if thisvalue > maxvalue:
        thisvalue = 0
        cur_label+=1
    newcum.append(thisvalue)
    labels.append(cur_label)
    lastvalue = thisvalue
to_divide_list['fcat']=labels
to_divide_list

,fname,fsize,row_perc,fcat
0,/datanaco/dharp/compounds/datasets/googleV3/80...,7677431026,1.550286e-01,1
1,/datanaco/dharp/compounds/datasets/googleV3/79...,7228628958,1.459660e-01,1
2,/datanaco/dharp/compounds/datasets/googleV3/80...,7134667418,1.440687e-01,1
3,/datanaco/dharp/compounds/datasets/googleV3/14...,7076838209,1.429009e-01,1
4,/datanaco/dharp/compounds/datasets/googleV3/81...,7050167074,1.423624e-01,1
...,...,...,...,...
6525,/datanaco/dharp/compounds/datasets/googleV3/16...,19672,3.972321e-07,100
6526,/datanaco/dharp/compounds/datasets/googleV3/13...,18182,3.671449e-07,100
6527,/datanaco/dharp/compounds/datasets/googleV3/16...,11446,2.311264e-07,100
6528,/datanaco/dharp/compounds/datasets/googleV3/22...,10357,2.091365e-07,100


In [35]:
to_divide_list.loc[to_divide_list.fcat.between(1,25)]

,fname,fsize,row_perc,fcat
0,/datanaco/dharp/compounds/datasets/googleV3/80...,7677431026,0.155029,1
1,/datanaco/dharp/compounds/datasets/googleV3/79...,7228628958,0.145966,1
2,/datanaco/dharp/compounds/datasets/googleV3/80...,7134667418,0.144069,1
3,/datanaco/dharp/compounds/datasets/googleV3/14...,7076838209,0.142901,1
4,/datanaco/dharp/compounds/datasets/googleV3/81...,7050167074,0.142362,1
...,...,...,...,...
197,/datanaco/dharp/compounds/datasets/googleV3/29...,5882970430,0.118793,25
198,/datanaco/dharp/compounds/datasets/googleV3/18...,5881815323,0.118770,25
199,/datanaco/dharp/compounds/datasets/googleV3/81...,5881468020,0.118763,25
200,/datanaco/dharp/compounds/datasets/googleV3/18...,5862185384,0.118374,25


In [36]:
to_divide_list.loc[to_divide_list.fcat.between(1,25)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/latinum_fcat.txt',header=True,index=False,sep="\t")

In [37]:
to_divide_list.loc[to_divide_list.fcat.between(26,50)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/mithril_fcat.txt',header=True,index=False,sep="\t")

In [38]:
to_divide_list.loc[to_divide_list.fcat.between(51,75)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/tiberium_fcat.txt',header=True,index=False,sep="\t")

In [39]:
to_divide_list.loc[to_divide_list.fcat.between(76,100)][['fname','fcat']].to_csv('/data/dharp/compounds/datasets/drogium_fcat.txt',header=True,index=False,sep="\t")